[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/0xInode/AI-study-notes/blob/main/ml/hands-on-ml/chapter09/chapter09.ipynb)


# 9. Unsupervised Learning Techniques

## Clustering

### K-Means

### Limits of K-Means

### Using Clustering for Image Segmentation

### Using Clustering for Preprocessing

### Using Clustering for Semi-Supervised Learning

### DBSCAN

### Other Clustering Algorithms

## Gaussian Mixtures

### Anomaly Detection Using Gaussian Mixtures

### Selecting the Number of Clusters

### Bayesian Gaussian Mixture Models

### Other Algorithms for Anomaly and Novelty Detection